In [1]:
# Al principio vamos a abrir una terminal para realizar observaciones rápidas
%qtconsole

### Pruebas de funciones

In [2]:
from Bio.Seq import Seq

cds_ref = Seq("ATGGATGTATTCATGAAAGGACTTTCAAAGGCCAAGGAGGGAGTTGTGGCTGCTGCTGAGAAAACCAAACAGGGTGTGGCAGAAGCAGCAGGAAAGACAAAAGAGGGTGTTCTCTATGTAGGCTCCAAAACCAAGGAGGGAGTGGTGCATGGTGTGGCAACAGTGGCTGAGAAGACCAAAGAGCAAGTGACAAATGTTGGAGGAGCAGTGGTGACGGGTGTGACAGCAGTAGCCCAGAAGACAGTGGAGGGAGCAGGGAGCATTGCAGCAGCCACTGGCTTTGTCAAAAAGGACCAGTTGGGCAAGGAAGGGTATCAAGACTACGAACCTGAAGCCTAA")
cds_loc = Seq("ATGGATGTATTCATGAAAGGACTTTCAAAGGCCAAGGAGGGAGTTGTGGCTGCTGCTGAGAAAACCAAACAGGGTGTGGCAGAAGCAGCAGGAAAGACAAAAGAGGGTGTTCTCTATGTAGGCTCCAAAACCAAGGAGGGAGTGGTGCATGGTGTGGCAACAGTGGCTGAGAAGACCAAAGAGCAAGTGACAAATGTTGGAGGAGCAGTGGTGACGGGTGTGACAGCAGTAGCCCAGAAGACAGTGGAGGGAGCAGGGAGCATTGCAGCAGCCACTGGCTTTGTCAAAAAGGACCAGTTGGGCAAGAATGAAGAAGGAGCCCCACAGGAAGGAATTCTGGAAGATATGCCTGTGGATCCTGACAATGAGGCTTATGAAATGCCTTCTGAGGAAGGGTATCAAGACTACGAACCTGAAGCCTAA")

print(len(cds_ref),len(cds_loc))


339 423


### GenBank records

In [45]:
#to: gbk_utilities.py
from Bio import SeqIO
import my_utilities as my_ut

file1="SNCA_ReferenceGRCh38_HomoSapiens.gb"
file2="SNCA_RefSeq_HomoSapiens.gb"

def report_cds(file):
    if my_ut.check_ext(file,"GenBank"):
        for rec in SeqIO.parse(file, "genbank"):
            if rec.features:
                cds=False
                for feature in rec.features:
                    if feature.type == "CDS":
                        cds=True
                        print("CDS Product: %s; ID: %s;" %(feature.qualifiers["product"][0],feature.qualifiers["protein_id"][0]))
                        i=0
                        for loc in feature.location.parts:
                            i+=1
                            print("Location exon %s: %s:%s" %(i,loc.start,loc.end))
                        print("\nSequence DNA: \n%s"%(feature.location.extract(rec).seq))
                        print("\nSequence Prot: \n%s"%(feature.location.extract(rec).seq.translate()))                    
                        print("------------------------------------------------------------")
                if not cds:
                    print("No CDS sequences were found")
        return

#Couldn't make this s**t work, spent more than 2 hours trying to figure out why it doesn't let me import my 
#modules. Keeps printing nameError and I honestly can't figure out why
#
#It seems like the problem is some incompatibility between jupyter and module importation
#def get_seqs_cds(file, seq_type:str, fasta=False, out_file=""):
#    if my_ut.hgdfhgdf(seq_type):
#        if my_ut.check_ext(file,"GenBank"):
#            if fasta:
#                if my_ut.check_ext(out_file,"FASTA"):
#                    o_file=open(out_file, "w")
#
#            for rec in SeqIO.parse(file, "genbank"):
#                if rec.features:
#                    cds=False
#                    for feature in rec.features:
#
#                        if feature.type == "CDS" and fasta:
#                            cds=True
#                            o_file.write(">%s %s" %(feature.qualifiers["product"][0],feature.qualifiers["protein_id"][0]))
#                            if seq_type == "rna":
#                                o_file.write("%s"%(feature.location.extract(rec).seq.transcribe()))
#                            elif seq_type == ("prot" or "protein"):
#                                o_file.write("%s"%(feature.location.extract(rec).seq.translate()))
#                            else:
#                                o_file.write("%s"%(feature.location.extract(rec).seq))
#
#                        if feature.type == "CDS" and not fasta:
#                            cds=True
#                            print(">%s %s" %(feature.qualifiers["product"][0],feature.qualifiers["protein_id"][0]))
#                            if seq_type == "rna":
#                                print("%s"%(feature.location.extract(rec).seq.transcribe()))
#                            elif seq_type == ("prot" or "protein"):
#                                print("%s"%(feature.location.extract(rec).seq.translate()))
#                            else:
#                                print("%s" %(feature.location.extract(rec).seq))
#
#                    if fasta:
#                    	o_file.close()
#
#                    if not cds:
#                        print("No CDS sequences were found")
#                    return
#####################################################################################################################


report_cds(file2)


CDS Product: alpha-synuclein isoform NACP140; ID: NP_000336.1;
Location exon 1: 1494:1615
Location exon 2: 8977:9019
Location exon 3: 14773:14916
Location exon 4: 107884:107968
Location exon 5: 110501:110534

Sequence DNA: 
ATGGATGTATTCATGAAAGGACTTTCAAAGGCCAAGGAGGGAGTTGTGGCTGCTGCTGAGAAAACCAAACAGGGTGTGGCAGAAGCAGCAGGAAAGACAAAAGAGGGTGTTCTCTATGTAGGCTCCAAAACCAAGGAGGGAGTGGTGCATGGTGTGGCAACAGTGGCTGAGAAGACCAAAGAGCAAGTGACAAATGTTGGAGGAGCAGTGGTGACGGGTGTGACAGCAGTAGCCCAGAAGACAGTGGAGGGAGCAGGGAGCATTGCAGCAGCCACTGGCTTTGTCAAAAAGGACCAGTTGGGCAAGAATGAAGAAGGAGCCCCACAGGAAGGAATTCTGGAAGATATGCCTGTGGATCCTGACAATGAGGCTTATGAAATGCCTTCTGAGGAAGGGTATCAAGACTACGAACCTGAAGCCTAA

Sequence Prot: 
MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA*
------------------------------------------------------------


In [48]:
#to: gbk_utilities.py
import re
import numpy as np
from Bio import SeqIO

file1= "SNCA_ReferenceGRCh38_HomoSapiens.gb"
file2="SNCA_RefSeq_HomoSapiens.gb"


def location_ori(file):
#This function will return us the location of our ORIGIN sequence of this GenBank file.
#Will work properly if there's only one ORIGIN (SeqRecord) per file
    for rec in SeqIO.parse(file, "genbank"):
        if rec.annotations:
            if rec.annotations['accessions']:
                locs = rec.annotations['accessions'][2]
    index = [1,3]
    #This index vector is created because the following 're.split()' will output a list
    #of strings with the structure: ['complement' '89724099' '' '89838324' '']
    #We also cast it to np.int_ afterwards
    locs = np.array(re.split("[().]", locs))[index]
    locs = list(np.int_(locs))
    return locs

print(location_ori(file1))
## NOTE: Make location_oris() later

[89724099, 89838324]


In [6]:
#to: gbk_utilities.py

file1 = "SNCA_ReferenceGRCh38_HomoSapiens.gb"

def get_cds_indx(file):
    if my_ut.check_ext(file,"GenBank"):        
        for rec in SeqIO.parse(file, "genbank"):            
            if rec.features:                
                cds=False
                cind_ls = list() #cds_index_list
                eind_ls = list() #exon_index_list
                i=0
                for feature in rec.features:                    
                    
                    if feature.type == "CDS":                        
                        cds=True                    
                        
                        #You might wanna look at the intervals each exon has with the omitted commands
                        #print("CDS Product: %s; ID: %s;" %(feature.qualifiers["product"][0],feature.qualifiers["protein_id"][0]))
                        
                        i=0
                        for loc in feature.location.parts:
                            i+=1
                            eind_ls = eind_ls + list(range(loc.start,loc.end))
                            
                            #print("Location exon %s: %s:%s" %(i,loc.start,loc.end))
                            #print("------------------------------------------------------------")
                        
                        cind_ls.append(eind_ls)                        
                if not cds:
                    print("No CDS sequences were found")
                    
        return cind_ls

print(get_cds_indx(file1))
###This funtion returns a list of lists, where each element is the location of each nucleotide in each
###exon in each cds-variant/isoform of the gene. This will come in handy in order to make logic vectors.

[[2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2684, 2685, 2686, 2687, 2688, 2689, 2690, 2691, 2692, 2693, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755, 2756, 2757, 2758, 2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766, 2767, 2768, 2769, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777, 10140, 10141, 10142, 10143, 10144, 10145, 10146, 10147, 10148, 10149, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10162, 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10177, 10178,

In [42]:
#to: snp_utilities.py
import numpy as np
#import gbk_utilities as gbk_u

#Please make sure your SNPs are from the same chromosome as the GenBank file you're compairing them with
file_snp = "SNP_Locations.txt"
file_gbk = "SNCA_ReferenceGRCh38_HomoSapiens.gb"
start_gb = location_ori(file_gbk)[0]
#The location of our GenBank file inside the chromosome
#Given that 'start_gb' is the first base inside our CDS sequence, then we would like to get the following
#result when snp were equal to start_gb: (snp=start_gb; snp-start_gb=1)
#Then, and for the sake of legibility we're just gonna rest 1 to start_gb
start_gb = start_gb - 1

def snps_list_parse(file, start_gb:int=0):
    locs = open(file)
    locl = str(locs.read()).split()

    snps = np.int_(locl)    #Here we change our file datatype to 'int' as well as we make it a numpy array
                            #which will make it easier to get their relative positions
    snps = snps - start_gb
    snps = snps[snps >= 1]  #Here we select only the SNPs whose location is at or after the start position
                            #in our GenBank file
    snps = snps.tolist()    #Then we cast the array back to list (optional)
    return snps

print(snps_list_parse(file_snp, start_gb))
len(snps_list_parse(file_snp, start_gb))

[5, 8, 15, 19, 20, 26, 27, 43, 44, 45, 56, 66, 72, 74, 76, 78, 79, 81, 88, 89, 98, 100, 112, 114, 115, 118, 121, 127, 133, 136, 138, 147, 150, 151, 152, 156, 157, 158, 159, 160, 166, 169, 172, 177, 179, 181, 184, 186, 199, 203, 204, 218, 227, 229, 233, 236, 248, 256, 257, 258, 259, 262, 265, 270, 273, 280, 287, 288, 294, 295, 298, 302, 308, 309, 310, 311, 312, 319, 327, 335, 343, 345, 347, 353, 354, 366, 367, 368, 370, 374, 376, 380, 381, 382, 383, 385, 392, 393, 396, 397, 398, 403, 404, 412, 417, 423, 426, 432, 435, 440, 443, 451, 454, 461, 463, 464, 465, 467, 469, 470, 471, 479, 488, 489, 494, 496, 498, 506, 508, 513, 515, 517, 524, 525, 527, 528, 532, 533, 535, 545, 552, 556, 562, 572, 574, 576, 577, 578, 583, 584, 585, 589, 593, 598, 600, 602, 606, 609, 610, 611, 612, 624, 625, 626, 633, 636, 642, 644, 646, 647, 650, 656, 660, 661, 663, 664, 666, 673, 674, 675, 677, 678, 684, 686, 687, 692, 693, 694, 697, 698, 709, 721, 728, 732, 738, 741, 753, 758, 760, 764, 765, 766, 768, 772, 77

16378

### Check if SNPs are inside

In [10]:
#to: snp_utilities.py 
import my_utilities as my_ut

file_gb = "SNCA_ReferenceGRCh38_HomoSapiens.gb"

file_snp = "SNP_Locations.txt"
start_gb = location_ori(file_gb)[0]
snp_ls = snps_list_parse(file_snp, start_gb)

def report_snp_in_gb(file,snps):
    if my_ut.check_ext(file, "GenBank"):
        for snp in snps:
            for rec in SeqIO.parse(file, "genbank"):
                if rec.features:
                    for feature in rec.features:
                        if feature.type == "CDS":
                            if snp in feature.location:
                                print("%s is in %s, ID: %s" %(snp,feature.qualifiers["product"][0],feature.qualifiers["protein_id"][0]))
    return


report_snp_in_gb(file_gb,snp_ls)

2657 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2657 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
2657 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
2657 is in alpha-synuclein isoform X1, ID: XP_011530505.1
2657 is in alpha-synuclein isoform X1, ID: XP_011530506.1
2657 is in alpha-synuclein isoform X1, ID: XP_011530507.1
2657 is in alpha-synuclein isoform X1, ID: XP_011530508.1
2657 is in alpha-synuclein isoform X1, ID: XP_011530509.1
2658 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2658 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2658 is in alpha-synuclein isoform N

2712 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2712 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
2712 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
2712 is in alpha-synuclein isoform X1, ID: XP_011530505.1
2712 is in alpha-synuclein isoform X1, ID: XP_011530506.1
2712 is in alpha-synuclein isoform X1, ID: XP_011530507.1
2712 is in alpha-synuclein isoform X1, ID: XP_011530508.1
2712 is in alpha-synuclein isoform X1, ID: XP_011530509.1
2715 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2715 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2715 is in alpha-synuclein isoform N

2757 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2757 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
2757 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
2757 is in alpha-synuclein isoform X1, ID: XP_011530505.1
2757 is in alpha-synuclein isoform X1, ID: XP_011530506.1
2757 is in alpha-synuclein isoform X1, ID: XP_011530507.1
2757 is in alpha-synuclein isoform X1, ID: XP_011530508.1
2757 is in alpha-synuclein isoform X1, ID: XP_011530509.1
2759 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
2759 is in alpha-synuclein isoform X2, ID: XP_016864052.1
2759 is in alpha-synuclein isoform N

10142 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
10142 is in alpha-synuclein isoform X2, ID: XP_016864052.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
10142 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
10142 is in alpha-synuclein isoform X1, ID: XP_011530505.1
10142 is in alpha-synuclein isoform X1, ID: XP_011530506.1
10142 is in alpha-synuclein isoform X1, ID: XP_011530507.1
10142 is in alpha-synuclein isoform X1, ID: XP_011530508.1
10142 is in alpha-synuclein isoform X1, ID: XP_011530509.1
10143 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
10143 is in alpha-synuclein isoform X2, ID: XP_016864052.1
10143 is in alpha-sy

15970 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
15970 is in alpha-synuclein isoform X2, ID: XP_016864052.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
15970 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
15970 is in alpha-synuclein isoform X1, ID: XP_011530505.1
15970 is in alpha-synuclein isoform X1, ID: XP_011530506.1
15970 is in alpha-synuclein isoform X1, ID: XP_011530507.1
15970 is in alpha-synuclein isoform X1, ID: XP_011530508.1
15970 is in alpha-synuclein isoform X1, ID: XP_011530509.1
15971 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
15971 is in alpha-synuclein isoform X2, ID: XP_016864052.1
15971 is in alpha-sy

16012 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
16012 is in alpha-synuclein isoform X2, ID: XP_016864052.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
16012 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
16012 is in alpha-synuclein isoform X1, ID: XP_011530505.1
16012 is in alpha-synuclein isoform X1, ID: XP_011530506.1
16012 is in alpha-synuclein isoform X1, ID: XP_011530507.1
16012 is in alpha-synuclein isoform X1, ID: XP_011530508.1
16012 is in alpha-synuclein isoform X1, ID: XP_011530509.1
16013 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
16013 is in alpha-synuclein isoform X2, ID: XP_016864052.1
16013 is in alpha-sy

16064 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
16064 is in alpha-synuclein isoform X2, ID: XP_016864052.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001362215.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001362217.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001362214.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001362216.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001139526.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_001139527.1
16064 is in alpha-synuclein isoform NACP140, ID: NP_000336.1
16064 is in alpha-synuclein isoform X1, ID: XP_011530505.1
16064 is in alpha-synuclein isoform X1, ID: XP_011530506.1
16064 is in alpha-synuclein isoform X1, ID: XP_011530507.1
16064 is in alpha-synuclein isoform X1, ID: XP_011530508.1
16064 is in alpha-synuclein isoform X1, ID: XP_011530509.1
16068 is in alpha-synuclein isoform NACP112, ID: NP_009292.1
16068 is in alpha-synuclein isoform X2, ID: XP_016864052.1
16068 is in alpha-sy

46734 is in alpha-synuclein isoform X1, ID: XP_011530505.1
46734 is in alpha-synuclein isoform X1, ID: XP_011530506.1
46734 is in alpha-synuclein isoform X1, ID: XP_011530507.1
46734 is in alpha-synuclein isoform X1, ID: XP_011530508.1
46734 is in alpha-synuclein isoform X1, ID: XP_011530509.1
46740 is in alpha-synuclein isoform X1, ID: XP_011530505.1
46740 is in alpha-synuclein isoform X1, ID: XP_011530506.1
46740 is in alpha-synuclein isoform X1, ID: XP_011530507.1
46740 is in alpha-synuclein isoform X1, ID: XP_011530508.1
46740 is in alpha-synuclein isoform X1, ID: XP_011530509.1
46744 is in alpha-synuclein isoform X1, ID: XP_011530505.1
46744 is in alpha-synuclein isoform X1, ID: XP_011530506.1
46744 is in alpha-synuclein isoform X1, ID: XP_011530507.1
46744 is in alpha-synuclein isoform X1, ID: XP_011530508.1
46744 is in alpha-synuclein isoform X1, ID: XP_011530509.1
46748 is in alpha-synuclein isoform X1, ID: XP_011530505.1
46748 is in alpha-synuclein isoform X1, ID: XP_011530506

In [43]:
import my_utilities as my_ut

file_gb = "SNCA_ReferenceGRCh38_HomoSapiens.gb"

file_snp = "SNP_Locations.txt"
start_gb = location_ori(file_gb)[0]
snp_ls = snps_list_parse(file_snp, start_gb)


def get_snp_in_gb(file,snps):
    snps_in_gb = []
    if my_ut.check_ext(file, "GenBank"):
        for rec in SeqIO.parse(file, "genbank"):
            if rec.features:
                for feature in rec.features:
                    if feature.type == "CDS":
                        for snp in snps:
                            if snp in feature.location:
                                if not snp in snps_in_gb:
                                    snps_in_gb.append(snp)
                                    continue
        return snps_in_gb

    
print(get_snp_in_gb(file_gb,snp_ls))
len(get_snp_in_gb(file_gb,snp_ls))
snp_in_cds = get_snp_in_gb(file_gb,snp_ls)


[2657, 2658, 2665, 2673, 2676, 2688, 2693, 2694, 2695, 2696, 2701, 2702, 2705, 2706, 2709, 2711, 2712, 2715, 2717, 2720, 2725, 2727, 2729, 2731, 2733, 2734, 2735, 2742, 2744, 2745, 2755, 2757, 2759, 2762, 2764, 2765, 2767, 2768, 2769, 2771, 2775, 10142, 10143, 10144, 10145, 10146, 10151, 10156, 10169, 10173, 15936, 15937, 15941, 15942, 15944, 15946, 15947, 15949, 15955, 15957, 15958, 15962, 15963, 15965, 15967, 15969, 15970, 15971, 15979, 15983, 15986, 15990, 15992, 15995, 15996, 15998, 15999, 16004, 16005, 16010, 16011, 16012, 16013, 16014, 16015, 16016, 16020, 16027, 16030, 16034, 16037, 16041, 16045, 16050, 16059, 16063, 16064, 16068, 16069, 16073, 16075, 46670, 46676, 46677, 46680, 46682, 46687, 46689, 46690, 46694, 46697, 46700, 46710, 46711, 46715, 46716, 46720, 46728, 46732, 46734, 46740, 46744, 46748, 46752, 46753, 46758, 46759, 46760, 46762, 46768, 46776, 46778, 46789, 46790, 46805, 46809, 46810, 46811, 46812, 46813, 46814, 46818, 46828, 46832, 46834, 46840, 46843, 46856, 4685

We've passed from 16k+ SNPs inside the CDS, to only 150 inside the coding region.

In [ ]:
#to snp_utilities
#assign values to each snp based on the number of alleles [2:4] 

file_snp = "SNP_Locations.txt"
file_alleles = "SNP_Alleles.txt"
snp_in_cds = w

